
# 1. Data Loading


In [1]:
# Read libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

from plotly.subplots import make_subplots
from datetime import datetime


In [2]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
# Define the path to the marketing data file
marketing_file = r'./case-study-task-marketing.parquet'

# Load the parquet file into a DataFrame
df = pd.read_parquet(marketing_file, engine="pyarrow")


# 2. Data Overview

- Use `.info()` to undestand the dtypes

- Use `.head()`. to understand structure and contents

- Use `.describe()`. to see descriptive statistics
    
- Look for null values

In [4]:
# Get a summary of the DataFrame (columns, non-null counts, data types)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  2627 non-null   object 
 1   platform              2627 non-null   object 
 2   marketing_channel     2627 non-null   object 
 3   campaign_id           2627 non-null   object 
 4   country               2627 non-null   object 
 5   impressions           2627 non-null   int64  
 6   clicks                2627 non-null   int64  
 7   app_installs          2627 non-null   int64  
 8   conversions           2627 non-null   int64  
 9   marketing_spend_euro  2627 non-null   float64
 10  revenue_euro          2627 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 225.9+ KB


In [5]:
# Display the first 5 rows of the DataFrame to understand its structure and contents
df.head()

,date,platform,marketing_channel,campaign_id,country,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro
0,2023-11-01,android,Channel A,12347578214,CH,0,0,3,1,0.00,12.94
1,2023-11-01,android,Channel A,12347578214,FR,70732,1116,469,37,688.56,915.76
2,2023-11-01,android,Channel A,12347578214,IT,0,0,1,0,0.00,0.00
3,2023-11-01,android,Channel A,12579890294,NL,17173,356,145,7,279.76,190.20
4,2023-11-01,android,Channel A,12581196474,FR,0,0,1,0,0.00,0.00


In [6]:
# Display descriptive statistics for numerical columns
print("## Descriptive Statistics:")
df.describe()

## Descriptive Statistics:


,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro
count,2627.000000,2627.000000,2627.000000,2627.000000,2627.000000,2627.000000
mean,9318.816521,90.460601,38.987438,3.604872,101.878489,101.513129
std,26912.639949,207.279428,105.853684,11.645432,225.837117,324.971323
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
50%,132.000000,9.000000,2.000000,0.000000,10.640000,0.000000
75%,3290.500000,94.000000,23.000000,2.000000,100.355000,46.790000
max,301149.000000,1498.000000,754.000000,133.000000,3248.770000,2983.160000


In [7]:
# Check for null values in the DataFrame
null_counts = df.isnull().sum()
print("## Null Values Count per Column:\n")
print(null_counts)

## Null Values Count per Column:

date                    0
platform                0
marketing_channel       0
campaign_id             0
country                 0
impressions             0
clicks                  0
app_installs            0
conversions             0
marketing_spend_euro    0
revenue_euro            0
dtype: int64


In [8]:
# Test to verify if there are no null values in the entire DataFrame
if df.isnull().sum().sum() == 0:
    print("## All clear! There are no null values in the DataFrame.")
else:
    raise ValueError("There are null values in the DataFrame!")

## All clear! There are no null values in the DataFrame.


> After understanding the dataframe, we can proceed to data transformation.

# 3. Data Transformation

Since we want to analyze this data from multiple dimensions (e.g. `country`,`platform `, `marketing_channel`, etc), it is convenient to write a function that can be reused flexibly.

I will follow the steps:
1. Aggregate the data using pandas `.agg()`
2. Calculate the metrics using `calculate_metrics()`
3. Style the output using `format_df()`

## 3.1  Calculate Metrics

The `calculate_metrics` function is important in marketing analytics because it ensures precise calculations by aggregating data before deriving metrics. For example, `CTR` is computed as the total clicks divided by the total impressions, avoiding errors that can arise from averaging individual `CTR` values.

These metrics represent key stages of the marketing funnel, providing insights into user behavior and campaign performance. Metrics are categorized into:

- **Absolute Metrics**: `impressions`, `clicks`, `conversions`...
- **Relative Conversion Metrics**: `CTR`, `click_to_install_rate`,`install_to_conversion`...
- **Associated Cost Metrics**: `CPC`, `CPI`...
- **Business Metrics**: `profit`, `ROAS`...

The function's reusable design makes it adaptable for analyzing different groupings, such as by campaign, platform, or time period, streamlining iterative analysis and enabling flexibility in decision-making.

In [9]:
# Function to Calculate Metrics
def calculate_metrics(df):
    """
    Calculates derived, cost, and business metrics for an aggregated DataFrame, replacing infinite values with NaN.

    Args:
        df (pd.DataFrame): Aggregated DataFrame with base columns.

    Returns:
        pd.DataFrame: DataFrame with additional calculated metrics.
    """
    # Create a copy to avoid modifying the input DataFrame
    df_metrics = df.copy()

    # Calculate derived/relative metrics
    df_metrics['click_through_rate'] = (df_metrics['clicks'] / df_metrics['impressions'] * 100).round(2)  # Click-through rate
    df_metrics['click_to_install_rate'] = (df_metrics['app_installs'] / df_metrics['clicks'] * 100).round(2)  # Install rate
    df_metrics['install_to_conversion_rate'] = (df_metrics['conversions'] / df_metrics['app_installs'] * 100).round(2)  # Conversion rate

    # Calculate cost-associated metrics
    df_metrics['cpm'] = (df_metrics['marketing_spend_euro'] / df_metrics['impressions'] * 1000).round(2)  # Cost per 1000 impressions
    df_metrics['cpc'] = (df_metrics['marketing_spend_euro'] / df_metrics['clicks']).round(2)  # Cost per click
    df_metrics['cpi'] = (df_metrics['marketing_spend_euro'] / df_metrics['app_installs']).round(2)  # Cost per install
    df_metrics['cpa'] = (df_metrics['marketing_spend_euro'] / df_metrics['conversions']).round(2)  # Cost per acquisition (conversion)

    # Calculate business metrics
    df_metrics['rpa'] = (df_metrics['revenue_euro'] / df_metrics['conversions']).round(2)  # Revenue per acquisition (conversion)
    df_metrics['profit'] = (df_metrics['revenue_euro'] - df_metrics['marketing_spend_euro']).round(2)  # Total profit
    df_metrics['return_on_ad_spend_percentage'] = (df_metrics['revenue_euro'] / df_metrics['marketing_spend_euro'] * 100).round(2)  # ROAS
    df_metrics['return_on_investment_percentage'] = (df_metrics['profit'] / df_metrics['marketing_spend_euro'] * 100).round(2)  # ROI

    # Replace infinite values with NaN to avoid problems when plotting
    df_metrics.replace([np.inf, -np.inf], np.nan, inplace=True)

    return df_metrics

## 3.2 Format metrics

In [10]:
# Function to Format Metrics for Display
def format_metrics(df):
    """
    Formats metrics for display using pandas Styler to apply consistent formatting, including percentages,
    currency, and thousands separators, while allowing visual enhancements like bars.

    Args:
        df (pd.DataFrame): DataFrame with calculated metrics.

    Returns:
        pd.io.formats.style.Styler: A styled DataFrame ready for display.
    """
    styled_df = df.style.hide(axis="index") \
        .bar(subset=['app_installs','conversions','marketing_spend_euro','revenue_euro','profit', 'cpi', 'install_to_conversion_rate'], align='mid', color=['#d65f5f', '#8ecbfa']) \
        .format(
            formatter={
                'impressions': "{:,}",
                'clicks': "{:,}",
                'app_installs': "{:,}",
                'conversions': "{:,}",
                'marketing_spend_euro': "€{:.2f}",
                'revenue_euro': "€{:.2f}",
                'click_through_rate': lambda x: f"{x:.2f}%",
                'click_to_install_rate': lambda x: f"{x:.2f}%",
                'install_to_conversion_rate': lambda x: f"{x:.2f}%",
                'cpm': "€{:.2f}",
                'cpc': "€{:.2f}",
                'cpi': "€{:.2f}",
                'cpa': "€{:.2f}",
                'profit': "€{:.2f}",
                'return_on_ad_spend_percentage': lambda x: f"{x:.2f}%",
                'return_on_investment_percentage': lambda x: f"{x:.2f}%",
                'rpa': "€{:.2f}"
            }
        )

    return styled_df

In [11]:
# Function to Format Metrics for Display, but for less column (the selected ones)
def format_metrics_selected_columns(df):
    """
    Formats metrics for display using pandas Styler to apply consistent formatting, including percentages,
    currency, and thousands separators, while allowing visual enhancements like bars.

    Args:
        df (pd.DataFrame): DataFrame with calculated metrics.

    Returns:
        pd.io.formats.style.Styler: A styled DataFrame ready for display.
    """
    styled_df = df.style.hide(axis="index") \
        .bar(subset=['app_installs', 'cpi', 'install_to_conversion_rate','profit'], align='mid', color=['#d65f5f', '#8ecbfa']) \
        .format(
            formatter={
                'impressions': "{:,}",
                'clicks': "{:,}",
                'app_installs': "{:,}",
                'conversions': "{:,}",
                'marketing_spend_euro': "€{:.2f}",
                'revenue_euro': "€{:.2f}",
                'click_through_rate': lambda x: f"{x:.2f}%",
                'click_to_install_rate': lambda x: f"{x:.2f}%",
                'install_to_conversion_rate': lambda x: f"{x:.2f}%",
                'cpm': "€{:.2f}",
                'cpc': "€{:.2f}",
                'cpi': "€{:.2f}",
                'cpa': "€{:.2f}",
                'profit': "€{:.2f}",
                'return_on_ad_spend_percentage': lambda x: f"{x:.2f}%",
                'return_on_investment_percentage': lambda x: f"{x:.2f}%",
                'rpa': "€{:.2f}"
            }
        )

    return styled_df

In [12]:
# absolute_columns = ['impressions', 'clicks', 'app_installs', 'conversions', 'marketing_spend_euro', 'revenue_euro']

# relative_columns = ['click_through_rate','click_to_install_rate', 'install_to_conversion_rate']

# cost_columns = ['cpm', 'cpc', 'cpi', 'cpa']

# business_columns =['profit', 'return_on_ad_spend_percentage','return_on_investment_percentage']

# main_columns =['marketing_spend_euro','cpa','profit', 'return_on_ad_spend_percentage']

# # format_df[main_columns]

## 3.3 Execute the transformations

```python
## SNIPPET ##
# Aggregate the data with the desired dimensions
agg_df = df.groupby([dimensions]).sum(numeric_only=True)

# Calculate the metrics
df_metrics = calculate_metrics(agg_df).reset_index()

# Filter the country
filtered_df_de = df_metrics[df_metrics['country'] == 'DE']
filtered_df_us = df_metrics[df_metrics['country'] == 'US']

# Stytle the output table
formatted_df = format_metrics(df_metrics)
```

### 3.3.1 Create `df_channels_over_time`

In [13]:
# Aggregate the data
agg_df = df.groupby(['country','marketing_channel','date']).sum(numeric_only=True)

#Calculate the metrics
df_metrics = calculate_metrics(agg_df).reset_index()

# Create the object
df_channels_over_time = df_metrics.copy()

# 4. Data Analysis

## 4.1 Main KPIS

In evaluating the performance of paid marketing campaigns, it is crucial to consider multiple Key Performance Indicators (KPIs) to gain a holistic understanding of campaign effectiveness. Each KPI provides unique insights into different aspects of performance, allowing for a balanced approach to decision-making. 

Since **the objective of these campaigns is to maximize app installations while maintaining a balanced consideration of financial implications**, keeping in mind the limitation to use few KPIs, I would choose: **Installs**, **Cost Per Install (CPI)**, **Install-to-Conversion Rate**, and **Profit**.

- **Installs** reflect the volume of the campaign, showing how many users have been acquired through paid marketing efforts.
- **CPI** reflects the cost-efficiency of acquiring app installs, addressing the primary objective of maximizing installations while considering financial constraints.
- **Install-to-Conversion Rate** evaluates the quality of installs by measuring how effectively they convert into paying subscribers, ensuring that the installs generate meaningful engagement.
- **Profit** captures the financial outcome of campaigns, balancing revenue generation against marketing expenses to highlight overall success.

In [14]:
selected_columns = ['country','platform','marketing_channel', 'app_installs','cpi','install_to_conversion_rate','profit']

In [15]:
agg_df = df.groupby(['country','platform','marketing_channel']).sum(numeric_only=True)

df_metrics = calculate_metrics(agg_df).reset_index()
selected_df = df_metrics[selected_columns].sort_values(['platform','profit'], ascending=False)

filtered_df_de = selected_df[selected_df['country'] == 'DE'].fillna(0)
filtered_df_us = selected_df[selected_df['country'] == 'US'].fillna(0)

# formatted_df = format_metrics(df_metrics)

display(format_metrics_selected_columns(filtered_df_de))

display(format_metrics_selected_columns(filtered_df_us))

country,platform,marketing_channel,app_installs,cpi,install_to_conversion_rate,profit
DE,ios,Channel C,"22,972",€1.59,13.71%,€51454.35
DE,ios,Channel E,225,€0.00,4.00%,€250.38
DE,ios,Channel A,3,€931.91,0.00%,€-2795.72
DE,ios,Channel B,15,€917.89,20.00%,€-13658.21
DE,android,Channel A,"17,951",€1.84,9.62%,€17064.63
DE,android,Channel B,"5,081",€3.12,4.43%,€-8593.46
DE,android,Channel E,"2,457",€5.86,7.41%,€-8626.30
DE,android,Channel D,0,€0.00,0.00%,€-12500.97


country,platform,marketing_channel,app_installs,cpi,install_to_conversion_rate,profit
US,ios,Channel E,0,€0.00,0.00%,€0.00
US,ios,Channel B,1,€2595.43,0.00%,€-2595.43
US,ios,Channel A,1,€3494.55,0.00%,€-3494.55
US,ios,Channel C,"3,590",€4.18,10.56%,€-5675.34
US,android,Channel A,6,€0.00,0.00%,€0.00
US,android,Channel E,0,€0.00,0.00%,€0.00
US,android,Channel D,0,€0.00,0.00%,€-1534.93
US,android,Channel B,"1,440",€3.97,7.29%,€-3181.17


#### **Germany Insights**

1. **Scale High-Performing Channels**:
   - Focus on **Channel C (iOS)** and **Channel A (Android)** for their profitability and efficiency(cpi).
2. **Reassess Poor Performers**:
   - Investigate high CPIs in **Channels A and B (iOS)** and optimize spending.
   - Reevaluate **Channel D (Android)** due to zero installs and total loss.

3. **Reduce Spending on Loss-Making Channels**:
   - Limit investments in **Channel B (Android)** and **Channel E (Android)** due to inefficiency.

#### **USA Insights**

1. **Scale High-Performing Channels**:
   - None of the channels in the USA are profitable, so there are no clear candidates for scaling at this point.
   - **Channel C (iOS)** shows potential with the highest install volume (3,590) and a decent install-to-conversion rate (10.56%). Efforts should focus on reducing CPI (€4.18) and improving profitability.

2. **Reassess Poor Performers**:
   - Investigate **Channel B (iOS)** and **Channel A (iOS)** due to extremely high CPIs (€2,595.43 and €3,494.55, respectively) and no conversions.
   - Reevaluate **Channel D (Android)**, which incurred a total loss (€-1,534.93) with zero installs.
   - Examine **Channel B (Android)**, which has a moderate install volume (1,440) but suffers from a loss (€-3,181.17) and a suboptimal conversion rate (7.29%).

3. **Reduce Spending on Loss-Making Channels**:
   - Limit investments in **Channels A and B (iOS)** due to inefficient spending and lack of conversions.
   - Deprioritize **Channels D and E (Android)**, as they generated no meaningful results.

4. **Opportunities for Improvement**:
   - Focus on increasing the conversion rates for **Channel C (iOS)** and **Channel B (Android)** to turn high install volumes into meaningful profit.
   - Explore alternative strategies for channels that consistently underperform across both platforms.

#### **Important to mention**
- There are €9560 of profit assigned to unknown country. Important to mention that this could be due to data problems, and part of this profit could be coming from Germany or US.
- Altough the marketing team is currently focused on DE and US, it is important to mention the economic losses in France are €18806, even bigger than US(€16481).

## 4.2 Complete Funnel

### 4.2.1 Performance by Channel


**Germany**
- Channels A and B with positive profit, install_to_conversion_rate > 0.10%, cost_per_install < 2 euros.
- The other Channels have negative profit.

**United States**
- All Channels with negative or inexistent profit, cost_per_install > 4 euros.
- Channel E has more clicks than impressions. There is a data quality problem.

In [16]:
agg_df = df.groupby(['country','marketing_channel']).sum(numeric_only=True)

df_metrics = calculate_metrics(agg_df).reset_index()

filtered_df_de = df_metrics[df_metrics['country'] == 'DE']
filtered_df_us = df_metrics[df_metrics['country'] == 'US']

# formatted_df = format_metrics(df_metrics)

display(format_metrics(filtered_df_de))

display(format_metrics(filtered_df_us))

country,marketing_channel,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro,click_through_rate,click_to_install_rate,install_to_conversion_rate,cpm,cpc,cpi,cpa,rpa,profit,return_on_ad_spend_percentage,return_on_investment_percentage
DE,Channel A,"2,069,078","42,008","17,954","1,727",€35760.32,€50029.23,2.03%,42.74%,9.62%,€17.28,€0.85,€1.99,€20.71,€28.97,€14268.91,139.90%,39.90%
DE,Channel B,"4,386,420","15,876","5,096",228,€29614.37,€7362.70,0.36%,32.10%,4.47%,€6.75,€1.87,€5.81,€129.89,€32.29,€-22251.67,24.86%,-75.14%
DE,Channel C,"104,013","36,691","22,972","3,150",€36512.65,€87967.00,35.28%,62.61%,13.71%,€351.04,€1.00,€1.59,€11.59,€27.93,€51454.35,240.92%,140.92%
DE,Channel D,0,0,0,0,€12500.97,€0.00,nan%,nan%,nan%,€nan,€nan,€nan,€nan,€nan,€-12500.97,0.00%,-100.00%
DE,Channel E,"6,922,072","14,971","2,682",191,€14400.22,€6024.30,0.22%,17.91%,7.12%,€2.08,€0.96,€5.37,€75.39,€31.54,€-8375.92,41.83%,-58.17%


country,marketing_channel,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro,click_through_rate,click_to_install_rate,install_to_conversion_rate,cpm,cpc,cpi,cpa,rpa,profit,return_on_ad_spend_percentage,return_on_investment_percentage
US,Channel A,"1,634,556","1,904",7,0,€3494.55,€0.00,0.12%,0.37%,0.00%,€2.14,€1.84,€499.22,€nan,€nan,€-3494.55,0.00%,-100.00%
US,Channel B,"789,069","4,031","1,441",105,€8309.59,€2532.99,0.51%,35.75%,7.29%,€10.53,€2.06,€5.77,€79.14,€24.12,€-5776.60,30.48%,-69.52%
US,Channel C,"78,645","8,056","3,590",379,€15001.15,€9325.81,10.24%,44.56%,10.56%,€190.75,€1.86,€4.18,€39.58,€24.61,€-5675.34,62.17%,-37.83%
US,Channel D,0,0,0,0,€1534.93,€0.00,nan%,nan%,nan%,€nan,€nan,€nan,€nan,€nan,€-1534.93,0.00%,-100.00%
US,Channel E,923,"11,775",0,0,€0.00,€0.00,1275.73%,0.00%,nan%,€0.00,€0.00,€nan,€nan,€nan,€0.00,nan%,nan%


### 4.2.2 Performance by Platform


**Germany**
- Similar volume of installs to android and ios
- ios profit: `€35250`
- android profit: `-€12656`

**United States**
- 70% of installs from ios, 30% from android
- ios profit: `-€11765`	
- android profit: `-€4716`	

In [17]:
agg_df = df.groupby(['country','platform']).sum(numeric_only=True)

df_metrics = calculate_metrics(agg_df).reset_index()

filtered_df_de = df_metrics[df_metrics['country'] == 'DE']
filtered_df_us = df_metrics[df_metrics['country'] == 'US']

# formatted_df = format_metrics(df_metrics)

display(format_metrics(filtered_df_de))

display(format_metrics(filtered_df_us))

country,platform,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro,click_through_rate,click_to_install_rate,install_to_conversion_rate,cpm,cpc,cpi,cpa,rpa,profit,return_on_ad_spend_percentage,return_on_investment_percentage
DE,android,"10,757,864","61,614","25,489","2,134",€75711.80,€63055.70,0.57%,41.37%,8.37%,€7.04,€1.23,€2.97,€35.48,€29.55,€-12656.10,83.28%,-16.72%
DE,ios,"2,723,719","47,932","23,215","3,162",€53076.73,€88327.53,1.76%,48.43%,13.62%,€19.49,€1.11,€2.29,€16.79,€27.93,€35250.80,166.41%,66.41%


country,platform,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro,click_through_rate,click_to_install_rate,install_to_conversion_rate,cpm,cpc,cpi,cpa,rpa,profit,return_on_ad_spend_percentage,return_on_investment_percentage
US,android,"330,401","13,962","1,446",105,€7249.09,€2532.99,4.23%,10.36%,7.26%,€21.94,€0.52,€5.01,€69.04,€24.12,€-4716.10,34.94%,-65.06%
US,ios,"2,172,792","11,804","3,592",379,€21091.13,€9325.81,0.54%,30.43%,10.55%,€9.71,€1.79,€5.87,€55.65,€24.61,€-11765.32,44.22%,-55.78%


### 4.2.3 Performance by Platform and Channel


**Germany**
- The profitability comes mainly from ios on Channel C

**United States**
- The channel with the highest spend has the highest loss.

In [18]:
agg_df = df.groupby(['country','platform','marketing_channel']).sum(numeric_only=True)

df_metrics = calculate_metrics(agg_df).reset_index()

filtered_df_de = df_metrics[df_metrics['country'] == 'DE']
filtered_df_us = df_metrics[df_metrics['country'] == 'US']

# formatted_df = format_metrics(df_metrics)

display(format_metrics(filtered_df_de))

display(format_metrics(filtered_df_us))

country,platform,marketing_channel,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro,click_through_rate,click_to_install_rate,install_to_conversion_rate,cpm,cpc,cpi,cpa,rpa,profit,return_on_ad_spend_percentage,return_on_investment_percentage
DE,android,Channel A,"1,627,339","38,801","17,951","1,727",€32964.60,€50029.23,2.38%,46.26%,9.62%,€20.26,€0.85,€1.84,€19.09,€28.97,€17064.63,151.77%,51.77%
DE,android,Channel B,"2,330,812","9,314","5,081",225,€15846.01,€7252.55,0.40%,54.55%,4.43%,€6.80,€1.70,€3.12,€70.43,€32.23,€-8593.46,45.77%,-54.23%
DE,android,Channel D,0,0,0,0,€12500.97,€0.00,nan%,nan%,nan%,€nan,€nan,€nan,€nan,€nan,€-12500.97,0.00%,-100.00%
DE,android,Channel E,"6,799,713","13,499","2,457",182,€14400.22,€5773.92,0.20%,18.20%,7.41%,€2.12,€1.07,€5.86,€79.12,€31.72,€-8626.30,40.10%,-59.90%
DE,ios,Channel A,"441,739","3,207",3,0,€2795.72,€0.00,0.73%,0.09%,0.00%,€6.33,€0.87,€931.91,€nan,€nan,€-2795.72,0.00%,-100.00%
DE,ios,Channel B,"2,055,608","6,562",15,3,€13768.36,€110.15,0.32%,0.23%,20.00%,€6.70,€2.10,€917.89,€4589.45,€36.72,€-13658.21,0.80%,-99.20%
DE,ios,Channel C,"104,013","36,691","22,972","3,150",€36512.65,€87967.00,35.28%,62.61%,13.71%,€351.04,€1.00,€1.59,€11.59,€27.93,€51454.35,240.92%,140.92%
DE,ios,Channel E,"122,359","1,472",225,9,€0.00,€250.38,1.20%,15.29%,4.00%,€0.00,€0.00,€0.00,€0.00,€27.82,€250.38,nan%,nan%


country,platform,marketing_channel,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro,click_through_rate,click_to_install_rate,install_to_conversion_rate,cpm,cpc,cpi,cpa,rpa,profit,return_on_ad_spend_percentage,return_on_investment_percentage
US,android,Channel A,0,0,6,0,€0.00,€0.00,nan%,nan%,0.00%,€nan,€nan,€0.00,€nan,€nan,€0.00,nan%,nan%
US,android,Channel B,"329,551","2,500","1,440",105,€5714.16,€2532.99,0.76%,57.60%,7.29%,€17.34,€2.29,€3.97,€54.42,€24.12,€-3181.17,44.33%,-55.67%
US,android,Channel D,0,0,0,0,€1534.93,€0.00,nan%,nan%,nan%,€nan,€nan,€nan,€nan,€nan,€-1534.93,0.00%,-100.00%
US,android,Channel E,850,"11,462",0,0,€0.00,€0.00,1348.47%,0.00%,nan%,€0.00,€0.00,€nan,€nan,€nan,€0.00,nan%,nan%
US,ios,Channel A,"1,634,556","1,904",1,0,€3494.55,€0.00,0.12%,0.05%,0.00%,€2.14,€1.84,€3494.55,€nan,€nan,€-3494.55,0.00%,-100.00%
US,ios,Channel B,"459,518","1,531",1,0,€2595.43,€0.00,0.33%,0.07%,0.00%,€5.65,€1.70,€2595.43,€nan,€nan,€-2595.43,0.00%,-100.00%
US,ios,Channel C,"78,645","8,056","3,590",379,€15001.15,€9325.81,10.24%,44.56%,10.56%,€190.75,€1.86,€4.18,€39.58,€24.61,€-5675.34,62.17%,-37.83%
US,ios,Channel E,73,313,0,0,€0.00,€0.00,428.77%,0.00%,nan%,€0.00,€0.00,€nan,€nan,€nan,€0.00,nan%,nan%


### 4.2.4 Performance by Country


- There is €9560 of profit assigned to unknown country. Important to mention that this could be due to data problems, and part of this profit could be coming from Germany or US.
- Altough the marketing team is currently focused on DE and US, it is important to mention the economic losses in France are €18806, even bigger than US(€16481).

In [19]:
agg_df = df.groupby(['country']).sum(numeric_only=True)

df_metrics = calculate_metrics(agg_df).reset_index()

display(format_metrics(df_metrics))

country,impressions,clicks,app_installs,conversions,marketing_spend_euro,revenue_euro,click_through_rate,click_to_install_rate,install_to_conversion_rate,cpm,cpc,cpi,cpa,rpa,profit,return_on_ad_spend_percentage,return_on_investment_percentage
AT,"2,412,052","14,168","5,553",506,€16083.78,€15680.84,0.59%,39.19%,9.11%,€6.67,€1.14,€2.90,€31.79,€30.99,€-402.94,97.49%,-2.51%
CH,"1,291,583","8,373","3,082",417,€13764.33,€21639.79,0.65%,36.81%,13.53%,€10.66,€1.64,€4.47,€33.01,€51.89,€7875.46,157.22%,57.22%
DE,"13,481,583","109,546","48,704","5,296",€128788.53,€151383.23,0.81%,44.46%,10.87%,€9.55,€1.18,€2.64,€24.32,€28.58,€22594.70,117.54%,17.54%
FR,"3,263,826","49,313","21,694","1,564",€57216.98,€38410.02,1.51%,43.99%,7.21%,€17.53,€1.16,€2.64,€36.58,€24.56,€-18806.96,67.13%,-32.87%
IT,"82,531","14,822","8,832",512,€8143.63,€9234.64,17.96%,59.59%,5.80%,€98.67,€0.55,€0.92,€15.91,€18.04,€1091.01,113.40%,13.40%
NL,"1,445,763","15,652","4,588",349,€15297.32,€8906.77,1.08%,29.31%,7.61%,€10.58,€0.98,€3.33,€43.83,€25.52,€-6390.55,58.22%,-41.78%
US,"2,503,193","25,766","5,038",484,€28340.22,€11858.80,1.03%,19.55%,9.61%,€11.32,€1.10,€5.63,€58.55,€24.50,€-16481.42,41.84%,-58.16%
unknown,0,0,"4,929",342,€0.00,€9560.90,nan%,nan%,6.94%,€nan,€nan,€0.00,€0.00,€27.96,€9560.90,nan%,nan%


## 4.3 Main KPIs over time

### Germany

In [20]:
### PREPARE THE DATA ###

# Aggregate the data
agg_df = df.groupby(['country','date']).sum(numeric_only=True)
#agg_df = df[df.marketing_channel == "Channel C"].groupby(['country','date']).sum(numeric_only=True) # filter channel c

#Calculate the metrics
df_metrics = calculate_metrics(agg_df).reset_index()

# Create the object
df_main_over_time = df_metrics.copy()

# Filter the country
df_filtered_plot = df_main_over_time[df_main_over_time.country=="DE"].copy()

# Ensure the 'date' column in the DataFrame is in datetime format
df_filtered_plot['date'] = pd.to_datetime(df_filtered_plot['date'])

# Create a subplot figure with 4 rows, 1 column
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,  # Align x-axes
    vertical_spacing=0.05  # Adjust spacing between plots
)

# Add traces for each metric
metrics = ['app_installs', 'cpi', 'install_to_conversion_rate', 'profit']
titles = [
    'App Installs',
    'Cost Per Install (CPI)',
    'Install to Conversion Rate (%)',
    'Profit (Euro)'
]

# Plot App Installs (Row 1)
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['app_installs'], mode='lines', name='App Installs'),
    row=1, col=1
)

# Plot CPI (Row 2) with y-axis starting at 0
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['cpi'], mode='lines', name='CPI'),
    row=2, col=1
)
fig.update_yaxes(range=[0, None], row=2, col=1)

# Plot Install to Conversion Rate (Row 3) with y-axis starting at 0
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['install_to_conversion_rate'], mode='lines', name='Install to Conversion Rate'),
    row=3, col=1
)
fig.update_yaxes(range=[0, None], row=3, col=1)

# Plot Profit (Row 4) as a bar chart
fig.add_trace(
    go.Bar(x=df_filtered_plot['date'], y=df_filtered_plot['profit'], name='Profit'),
    row=4, col=1
)

# Add a vertical line and text annotation for Black Friday
black_friday_date = datetime(2023, 11, 24)  # Use a raw datetime object

# Add a shape for the vertical line
fig.add_shape(
    type="line",
    x0=black_friday_date, x1=black_friday_date,
    y0=0, y1=1,  # Relative to the y-axis (scaled by domain)
    xref="x", yref="paper",  # 'paper' ensures it spans all plots
    line=dict(color="red", width=2, dash="dash")
)

# Add a text annotation
fig.add_annotation(
    x=black_friday_date,
    y=1,  # Place it at the top of the plot area
    text="Black Friday",
    showarrow=False,
    xref="x", yref="paper",
    font=dict(size=12, color="red"),
    align="center"
)

# Update layout for better appearance
fig.update_layout(
    height=800,  # Adjust height
    title_text="Metrics Over Time - Germany",
    showlegend=False,  # Hide legend for clarity
)

# Update individual axes titles
fig.update_yaxes(title_text="App Installs", row=1, col=1)
fig.update_yaxes(title_text="CPI (Euro)", row=2, col=1)
fig.update_yaxes(title_text="Install to Conversion Rate (%)", row=3, col=1)
fig.update_yaxes(title_text="Profit (Euro)", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=1)

# Show the plot
fig.show()


- There is an increase in installs after black friday.
- The cost per install is higher 1 week before black friday.
- The install to conversion rate increases at black friday, probably because of a good offer.
- The beginning of the month is not profitable.
- There is a CPI max on 19/11 that contributes to a profit minimum.

### Germany - Specific Channel

In [21]:
### PREPARE THE DATA ###

# Aggregate the data
agg_df = df[df.marketing_channel == "Channel C"].groupby(['country','date']).sum(numeric_only=True)

#Calculate the metrics
df_metrics = calculate_metrics(agg_df).reset_index()

# Create the object
df_main_over_time = df_metrics.copy()


# Filter the country
df_filtered_plot = df_main_over_time[df_main_over_time.country=="DE"].copy()

# Ensure the 'date' column in the DataFrame is in datetime format
df_filtered_plot['date'] = pd.to_datetime(df_filtered_plot['date'])

### PLOT ###

# Create a subplot figure with 4 rows, 1 column
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,  # Align x-axes
    vertical_spacing=0.05  # Adjust spacing between plots
)

# Add traces for each metric
metrics = ['app_installs', 'cpi', 'install_to_conversion_rate', 'profit']
titles = [
    'App Installs',
    'Cost Per Install (CPI)',
    'Install to Conversion Rate (%)',
    'Profit (Euro)'
]

# Plot App Installs (Row 1)
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['app_installs'], mode='lines', name='App Installs'),
    row=1, col=1
)

# Plot CPI (Row 2) with y-axis starting at 0
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['cpi'], mode='lines', name='CPI'),
    row=2, col=1
)
fig.update_yaxes(range=[0, None], row=2, col=1)

# Plot Install to Conversion Rate (Row 3) with y-axis starting at 0
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['install_to_conversion_rate'], mode='lines', name='Install to Conversion Rate'),
    row=3, col=1
)
fig.update_yaxes(range=[0, None], row=3, col=1)

# Plot Profit (Row 4) as a bar chart
fig.add_trace(
    go.Bar(x=df_filtered_plot['date'], y=df_filtered_plot['profit'], name='Profit'),
    row=4, col=1
)

# Add a vertical line and text annotation for Black Friday
black_friday_date = datetime(2023, 11, 24)  # Use a raw datetime object

# Add a shape for the vertical line
fig.add_shape(
    type="line",
    x0=black_friday_date, x1=black_friday_date,
    y0=0, y1=1,  # Relative to the y-axis (scaled by domain)
    xref="x", yref="paper",  # 'paper' ensures it spans all plots
    line=dict(color="red", width=2, dash="dash")
)

# Add a text annotation
fig.add_annotation(
    x=black_friday_date,
    y=1,  # Place it at the top of the plot area
    text="Black Friday",
    showarrow=False,
    xref="x", yref="paper",
    font=dict(size=12, color="red"),
    align="center"
)

# Update layout for better appearance
fig.update_layout(
    height=900,  # Adjust height
    title_text="Metrics Over Time - Germany - Channel C",
    showlegend=False,  # Hide legend for clarity
)

# Update individual axes titles
fig.update_yaxes(title_text="App Installs", row=1, col=1)
fig.update_yaxes(title_text="CPI (Euro)", row=2, col=1)
fig.update_yaxes(title_text="Install to Conversion Rate (%)", row=3, col=1)
fig.update_yaxes(title_text="Profit (Euro)", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=1)

# Show the plot
fig.show()


### United States

In [22]:
# Filter the country
df_filtered_plot = df_main_over_time[df_main_over_time.country=="US"].copy()

# Ensure the 'date' column in the DataFrame is in datetime format
df_filtered_plot['date'] = pd.to_datetime(df_filtered_plot['date'])

# Create a subplot figure with 4 rows, 1 column
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,  # Align x-axes
    vertical_spacing=0.05  # Adjust spacing between plots
)

# Add traces for each metric
metrics = ['app_installs', 'cpi', 'install_to_conversion_rate', 'profit']
titles = [
    'App Installs',
    'Cost Per Install (CPI)',
    'Install to Conversion Rate (%)',
    'Profit (Euro)'
]

# Plot App Installs (Row 1)
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['app_installs'], mode='lines', name='App Installs'),
    row=1, col=1
)

# Plot CPI (Row 2) with y-axis starting at 0
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['cpi'], mode='lines', name='CPI'),
    row=2, col=1
)
fig.update_yaxes(range=[0, None], row=2, col=1)

# Plot Install to Conversion Rate (Row 3) with y-axis starting at 0
fig.add_trace(
    go.Scatter(x=df_filtered_plot['date'], y=df_filtered_plot['install_to_conversion_rate'], mode='lines', name='Install to Conversion Rate'),
    row=3, col=1
)
fig.update_yaxes(range=[0, None], row=3, col=1)

# Plot Profit (Row 4) as a bar chart
fig.add_trace(
    go.Bar(x=df_filtered_plot['date'], y=df_filtered_plot['profit'], name='Profit'),
    row=4, col=1
)

# Add a vertical line and text annotation for Black Friday
black_friday_date = datetime(2023, 11, 24)  # Use a raw datetime object

# Add a shape for the vertical line
fig.add_shape(
    type="line",
    x0=black_friday_date, x1=black_friday_date,
    y0=0, y1=1,  # Relative to the y-axis (scaled by domain)
    xref="x", yref="paper",  # 'paper' ensures it spans all plots
    line=dict(color="red", width=2, dash="dash")
)

# Add a text annotation
fig.add_annotation(
    x=black_friday_date,
    y=1,  # Place it at the top of the plot area
    text="Black Friday",
    showarrow=False,
    xref="x", yref="paper",
    font=dict(size=12, color="red"),
    align="center"
)

# Update layout for better appearance
fig.update_layout(
    height=900,  # Adjust height
    title_text="Metrics Over Time - United States",
    showlegend=False,  # Hide legend for clarity
)

# Update individual axes titles
fig.update_yaxes(title_text="App Installs", row=1, col=1)
fig.update_yaxes(title_text="CPI (Euro)", row=2, col=1)
fig.update_yaxes(title_text="Install to Conversion Rate (%)", row=3, col=1)
fig.update_yaxes(title_text="Profit (Euro)", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=1)

# Show the plot
fig.show()


- There is an drop on installs on the last 15 days.
- The cost per install is relatively stable, with a peak one day before black friday.
- The conversion rate has high variability.
- The profit is negative in all days but 5th and 6th of November

### By channel - Germany

In [23]:
import plotly.graph_objects as go

def create_grouped_line_chart(df, x_column, y_column, group_column, y_label, width=1000, height=600):
    """
    Create a grouped line chart with multiple lines and an annotation for Black Friday.

    Args:
        df (pd.DataFrame): DataFrame containing the data to plot.
        x_column (str): Name of the column for the x-axis.
        y_column (str): Name of the column for the y-axis.
        group_column (str): Name of the column to differentiate the lines.
        y_label (str): Label for the y-axis.
        width (int): Width of the figure (default: 1000).
        height (int): Height of the figure (default: 600).

    Returns:
        None
    """
    title = f"{y_column.replace('_', ' ').capitalize()} Over Time by {group_column.replace('_', ' ').capitalize()}"

    fig = go.Figure()

    # Create a line for each group
    for group, group_data in df.groupby(group_column):
        fig.add_trace(go.Scatter(
            x=group_data[x_column],
            y=group_data[y_column],
            mode='lines',
            name=group  # Use the group name for legend
        ))

    # Add annotation for Black Friday
    fig.add_annotation(
        x="2023-11-24",
        y=df[y_column].max(),
        text="Black Friday",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )

    # Customize layout
    fig.update_layout(
        title=title,
        xaxis=dict(title=x_column.capitalize()),
        yaxis=dict(title=y_label),
        legend=dict(title=group_column.replace('_', ' ').capitalize()),
        width=width,
        height=height
    )

    fig.show()


In [24]:
agg_df = df.groupby(['country','platform','marketing_channel','date']).sum(numeric_only=True)

df_metrics = calculate_metrics(agg_df).reset_index()

df_metrics['platform_and_channel'] = df_metrics['platform'] + "_" + df_metrics['marketing_channel']

In [25]:

# Example Usage
create_grouped_line_chart(
    df=df_channels_over_time[df_channels_over_time['country']=='DE'],
    x_column="date",
    y_column="app_installs",
    group_column="marketing_channel",
    y_label="App Installs"
)


In [26]:
create_grouped_line_chart(
    df=df_channels_over_time[df_channels_over_time['country']=='DE'],
    x_column="date",
    y_column="install_to_conversion_rate",
    group_column="marketing_channel",
    y_label="Install to Conversion rate"
)


In [27]:
create_grouped_line_chart(
    df=df_channels_over_time[df_channels_over_time['country']=='DE'],
    x_column="date",
    y_column="cpi",
    group_column="marketing_channel",
    y_label="Cost per Install"
)

In [28]:
create_grouped_line_chart(
    df=df_channels_over_time[df_channels_over_time['country']=='DE'],
    x_column="date",
    y_column="profit",
    group_column="marketing_channel",
    y_label="Profit"
)

### By channel - US

In [29]:

# Example Usage
create_grouped_line_chart(
    df=df_channels_over_time[df_channels_over_time['country']=='US'],
    x_column="date",
    y_column="app_installs",
    group_column="marketing_channel",
    y_label="App Installs"
)


In [30]:
create_grouped_line_chart(
    df=df_channels_over_time[df_channels_over_time['country']=='US'],
    x_column="date",
    y_column="install_to_conversion_rate",
    group_column="marketing_channel",
    y_label="Install to Conversion rate"
)


In [31]:
create_grouped_line_chart(
    df=df_channels_over_time[df_channels_over_time['country']=='US'],
    x_column="date",
    y_column="cpi",
    group_column="marketing_channel",
    y_label="Cost per Install"
)

In [32]:
create_grouped_line_chart(
    df=df_channels_over_time[df_channels_over_time['country']=='US'],
    x_column="date",
    y_column="profit",
    group_column="marketing_channel",
    y_label="Profit"
)

## Other KPIs over time

### Germany

##### Economy over time

In [33]:
# Plot marketing_spend_euro, revenue_euro, and profit over date

def plot_metrics(df, plot_width=1000, plot_height=600):
    fig = go.Figure()

    # Add lines for each metric with German palette colors
    fig.add_trace(go.Scatter(x=df['date'], y=df['marketing_spend_euro'], mode='lines', name='Marketing Spend (€)', line=dict(color='#fc5c65')))
    fig.add_trace(go.Scatter(x=df['date'], y=df['revenue_euro'], mode='lines', name='Revenue (€)', line=dict(color='#26de81')))
    fig.add_trace(go.Scatter(x=df['date'], y=df['profit'], mode='lines', name='Profit (€)', line=dict(color='#4b7bec')))

    # Add annotation for Black Friday on 24/11/2023
    fig.add_annotation(
        x="2023-11-24", 
        y=max(df['marketing_spend_euro'].max(), df['revenue_euro'].max()),  # Position at the top of the plot
        text="Black Friday",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )
    
    # Customize layout
    fig.update_layout(
        title="Marketing Spend, Revenue, and Profit over Time (DE)",
        xaxis_title="Date",
        yaxis_title="Amount (€)",
        width=plot_width,
        height=plot_height,
        legend=dict(title="Metrics")
    )

    fig.show()

In [34]:
plot_metrics(df_main_over_time[df_main_over_time.country=="DE"])

#### CPA, RPA, Conversion over date

In [35]:
# Plot cpa, rpa, and conversions over date

def plot_cpa_rpa_spend(df, plot_width=700, plot_height=500):
    fig = go.Figure()

    # Add lines for CPA and RPA with primary y-axis
    fig.add_trace(go.Scatter(x=df['date'], y=df['cpa'], mode='lines', name='CPA (€)', line=dict(color='#fc5c65'), yaxis='y1'))
    fig.add_trace(go.Scatter(x=df['date'], y=df['rpa'], mode='lines', name='RPA (€)', line=dict(color='#26de81'), yaxis='y1'))

    # Add conversions as a transparent bar chart with secondary y-axis
    fig.add_trace(go.Bar(x=df['date'], y=df['conversions'], name='Conversions', marker=dict(color='rgba(75, 126, 236, 0.6)'), yaxis='y2'))

    # Add annotation for Black Friday on 24/11/2023
    fig.add_annotation(
        x="2023-11-24", 
        y=max(df['cpa'].max(), df['rpa'].max()),  # Position at the top of the plot
        text="Black Friday",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )

    # Customize layout with dual y-axes
    fig.update_layout(
        title="CPA, RPA, and Conversions over Time",
        xaxis_title="Date",
        yaxis_title="CPA and RPA (€)",
        yaxis2=dict(
            title="Conversions",
            overlaying="y",
            side="right",
        ),
        barmode='group',
        width=plot_width,
        height=plot_height,
        legend=dict(title="Metrics")
    )

    fig.show()
    

# Plot metrics for DE
plot_cpa_rpa_spend(df_main_over_time[df_main_over_time.country=="DE"])

- There is an drop in RPA after black friday, but it is probably due to an offer.

### United States

##### Economy over time

In [36]:
# Plot marketing_spend_euro, revenue_euro, and profit over date

def plot_metrics(df, plot_width=1000, plot_height=600):
    fig = go.Figure()

    # Add lines for each metric with German palette colors
    fig.add_trace(go.Scatter(x=df['date'], y=df['marketing_spend_euro'], mode='lines', name='Marketing Spend (€)', line=dict(color='#fc5c65')))
    fig.add_trace(go.Scatter(x=df['date'], y=df['revenue_euro'], mode='lines', name='Revenue (€)', line=dict(color='#26de81')))
    fig.add_trace(go.Scatter(x=df['date'], y=df['profit'], mode='lines', name='Profit (€)', line=dict(color='#4b7bec')))

    # Add annotation for Black Friday on 24/11/2023
    fig.add_annotation(
        x="2023-11-24", 
        y=max(df['marketing_spend_euro'].max(), df['revenue_euro'].max()),  # Position at the top of the plot
        text="Black Friday",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )
    
    # Customize layout
    fig.update_layout(
        title="Marketing Spend, Revenue, and Profit over Time (US)",
        xaxis_title="Date",
        yaxis_title="Amount (€)",
        width=plot_width,
        height=plot_height,
        legend=dict(title="Metrics")
    )

    fig.show()

In [37]:
plot_metrics(df_main_over_time[df_main_over_time.country=="US"])

#### CPA, RPA, Conversion over date

In [38]:
# Plot cpa, rpa, and conversions over date

def plot_cpa_rpa_spend(df, plot_width=1000, plot_height=600):
    fig = go.Figure()

    # Add lines for CPA and RPA with primary y-axis
    fig.add_trace(go.Scatter(x=df['date'], y=df['cpa'], mode='lines', name='CPA (€)', line=dict(color='#fc5c65'), yaxis='y1'))
    fig.add_trace(go.Scatter(x=df['date'], y=df['rpa'], mode='lines', name='RPA (€)', line=dict(color='#26de81'), yaxis='y1'))

    # Add conversions as a transparent bar chart with secondary y-axis
    fig.add_trace(go.Bar(x=df['date'], y=df['conversions'], name='Conversions', marker=dict(color='rgba(75, 126, 236, 0.6)'), yaxis='y2'))

    # Add annotation for Black Friday on 24/11/2023
    fig.add_annotation(
        x="2023-11-24", 
        y=max(df['cpa'].max(), df['rpa'].max()),  # Position at the top of the plot
        text="Black Friday",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40
    )

    # Customize layout with dual y-axes
    fig.update_layout(
        title="CPA, RPA, and Conversions over Time",
        xaxis_title="Date",
        yaxis_title="CPA and RPA (€)",
        yaxis2=dict(
            title="Conversions",
            overlaying="y",
            side="right",
        ),
        barmode='group',
        width=plot_width,
        height=plot_height,
        legend=dict(title="Metrics")
    )

    fig.show()

# Plot metrics for DE
plot_cpa_rpa_spend(df_main_over_time[df_main_over_time.country=="US"])

- The RPA is always lower than CPA, leading to negative profitability.